# Model Implementation and Learning

This notebook is intended for the the main function operations for the projects, including function for preprocessing, learning and prediction results. Toward the end of the notebook we run the code on the imported dataset. The results are then stored locally. Note, the run time may take time,it took circa 8 hours on my personal PC.

### Table of code blocks

[1) Preprocessing](#1)

[2) LSTM](#2)

[3) Attention Model](#3)

[4) RNN](#4)

[5) GRU](#5)

[6) CKCNN](#6)

[7) Results and Prediction Generators](#7)

[8) Data Learner and Optimizer](#8)

In [148]:

import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, GRU, SimpleRNN,Flatten, Dense
from tensorflow.keras.layers import Reshape,  AdditiveAttention, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.utils import plot_model
from tensorflow.keras import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.random import set_seed

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils import weight_norm
import ckconv

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

set_seed(10)
torch.cuda.is_available()

True

In [248]:
# Global variables

T =10 #timesteps
ratio = 0.5 # train/test ratio
input_D = 5 #input dimension
output_D = 1 #output_dimension
df = pd.read_csv("SP 500 Stock Prices 2014-2017.csv").sort_values(by  = "symbol") # data



## Preprocessing <a name="1"></a>

In [150]:
# preprocess the data according to the relevant timestep offset
def preprocessor(data, target_offset ):
    scaler_list=[]
    n_seq = int((len(data) - (T + target_offset - 1) ))
    X = np.zeros((n_seq, T , input_D))
    Y = np.zeros((n_seq  ,  output_D))
    X_unscaled = np.zeros((n_seq  , T , input_D))
    Y_unscaled = np.zeros((n_seq  , output_D))

    #use min-max scaler to scale the data
    for t in range(n_seq):
        input_point = data[t:t+T+target_offset]
        scaler = MinMaxScaler().fit(input_point)
        scaler_list.append(scaler)
        scaled_input_point =scaler.transform(input_point)
        feature_inputs = scaled_input_point[:-target_offset]    
        target_outputs = scaled_input_point[-1:,3:4][0][0]
        feature_inputs_unscaled = input_point[:-target_offset]
        targt_outputs_unscaled = input_point[-1:,3:4]

        X[t,:,:] = feature_inputs 
        Y[t,:]=target_outputs
        X_unscaled[t,:,:] = feature_inputs_unscaled
        Y_unscaled[t,:] = targt_outputs_unscaled
                
    return X, Y , X_unscaled, Y_unscaled, scaler_list
        

In [151]:
# split the data to train and test according to the given train-test ratio
# consider both scaled data and unscaled
def trainTestSplit(ratio,X, Y , X_unscaled, Y_unscaled):

    n_train  =  int(len(X) * ratio  )
    X_train = X[:n_train]
    Y_train = Y[:n_train]
    X_test = X[n_train:]
    Y_test = Y[n_train:]
    
    # unscaled - raw data
    X_train_unscaled = X_unscaled[:n_train]
    Y_train_unscaled = Y_unscaled[:n_train]
    X_test_unscaled = X_unscaled[n_train:]
    Y_test_unscaled = Y_unscaled[n_train:]
    
    return X_train,Y_train, X_test, Y_test, X_train_unscaled, Y_train_unscaled, X_test_unscaled, Y_test_unscaled

## LSTM Implementation <a name="2"></a>

In [152]:
def modelLSTM(epochs: int,  X_train, Y_train , X_test, Y_test,lr,units,batch , verbose, callback,plot_summary = True):

    i = Input(shape=(T, input_D))
    x = LSTM(units)(i)
    x = Dense(output_D)(x)

    model = Model(i, x)
    model.compile(
      loss='mse',
      optimizer=Adam(learning_rate=lr),)
    
    if plot_summary == True:
        print(model.summary())
    
    if callback==True:
        callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)
        r = model.fit(X_train, Y_train, epochs=epochs,
                      validation_data=(X_test, Y_test),
                      callbacks=[callback], verbose=verbose,batch_size=batch)
    else:
        r = model.fit(X_train, Y_train, epochs=epochs,
                      validation_data=(X_test, Y_test),verbose=verbose,batch_size=batch)        
            
    return r,model

## Attention Models <a name="3"></a>

In [153]:
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super().__init__()
    self.W1 = tf.keras.layers.Dense(units, use_bias=False)
    self.W2 = tf.keras.layers.Dense(units, use_bias=False)
    self.attention = tf.keras.layers.AdditiveAttention()

  def call(self, query, value):

    # From Eqn. (4), `W1@ht`.
    w1_query = self.W1(query)
    # From Eqn. (4), `W2@hs`.
    w2_key = self.W2(value)

    query_mask = tf.ones(tf.shape(query)[:-1], dtype=bool)
    value_mask = tf.ones(tf.shape(value)[:-1], dtype=bool)

    context_vector = self.attention(
        inputs = [w1_query, value, w2_key],
        mask=[query_mask, value_mask],)
    
    return context_vector

In [154]:
def attentionGRU(epochs: int,  X_train, Y_train , X_test, Y_test,lr,units,batch , verbose, callback,plot_summary = True):

    i = Input(shape=(T, input_D))
    h_t,h_s = GRU(units,return_sequences = True, return_state = True)(i)
    attention_layer = BahdanauAttention(units)
    context_vector = attention_layer(query=h_t,value=h_s)
    context_and_rnn_output = tf.concat([context_vector, h_t], axis=-1)
    context_and_rnn_output = Flatten()(context_and_rnn_output)
    output = Dense(1)(context_and_rnn_output )

    model = Model(i, output)
    model.compile(
      loss='mse',
      optimizer=Adam(learning_rate=lr),)
    
    if plot_summary == True:
        print(model.summary())
    
    if callback==True:
        callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)
        r = model.fit(X_train, Y_train, epochs=epochs,
                      validation_data=(X_test, Y_test),
                      callbacks=[callback], verbose=verbose,batch_size=batch)
    else:
        r = model.fit(X_train, Y_train, epochs=epochs,
                      validation_data=(X_test, Y_test),verbose=verbose,batch_size=batch)  
    
    return r,model

## RNN Implementation <a name="4"></a>

In [155]:
def modelRNN(epochs: int,  X_train, Y_train , X_test, Y_test,lr,units,batch , verbose, callback,plot_summary = True):

    i = Input(shape=(T, input_D))
    x = SimpleRNN(units)(i)
    x = Dense(output_D)(x)
    model = Model(i, x)
    model.compile(
      loss='mse',
      optimizer=Adam(learning_rate=lr),)
    
    if plot_summary == True:
        print(model.summary())
    
    if callback==True:
        callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)
        r = model.fit(X_train, Y_train, epochs=epochs,
                      validation_data=(X_test, Y_test),
                      callbacks=[callback], verbose=verbose,batch_size=batch)
    else:
        r = model.fit(X_train, Y_train, epochs=epochs,
                      validation_data=(X_test, Y_test),verbose=verbose,batch_size=batch)      
    return r,model

## GRU Implementation  <a name="5"></a>

In [156]:
def modelGRU(epochs: int,  X_train, Y_train , X_test, Y_test,lr,units,batch , verbose, callback,plot_summary = True):

    i = Input(shape=(T, input_D))
    x = GRU(units)(i)
    x = Dense(output_D)(x)
    model = Model(i, x)
    model.compile(
      loss='mse',
      optimizer=Adam(learning_rate=lr),)
    
    if plot_summary == True:
        print(model.summary())
    
    if callback==True:
        callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)
        r = model.fit(X_train, Y_train, epochs=epochs,
                      validation_data=(X_test, Y_test),
                      callbacks=[callback], verbose=verbose,batch_size=batch)
    else:
        r = model.fit(X_train, Y_train, epochs=epochs,
                      validation_data=(X_test, Y_test),verbose=verbose,batch_size=batch)  
    
    return r,model

## CKCNN Implementation  <a name="6"></a>

In [160]:
device = torch.device("cuda:0")
class CKCNN_backbone(torch.nn.Module):
    def __init__(
        self,
        in_channels: int,
        hidden_channels: int,
        num_blocks: int,
        kernelnet_hidden_channels: int,
        kernelnet_activation_function: str,
        kernelnet_norm_type: str,
        dim_linear: int,
        bias: bool,
        omega_0: bool,
        dropout: float,
        weight_dropout: float,
    ):
        super().__init__()
        
        # Add num_blocks CKBlocks to a sequential called self.backbone
        blocks = []
        for i in range(num_blocks):
            block_in_channels = in_channels if i == 0 else hidden_channels
            blocks.append(
                ckconv.nn.CKBlock(
                    block_in_channels,
                    hidden_channels,
                    kernelnet_hidden_channels,
                    kernelnet_activation_function,
                    kernelnet_norm_type,
                    dim_linear,
                    bias,
                    omega_0,
                    dropout,
                    weight_dropout,
                )
            )
        self.backbone = torch.nn.Sequential(*blocks)

    def forward(self, x):
        return self.backbone(x)

In [161]:
class CKCNN(CKCNN_backbone):
    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        hidden_channels: int,
        num_blocks: int,
        kernelnet_hidden_channels: int,
        kernelnet_activation_function: str,
        kernelnet_norm_type: str,
        dim_linear: int,
        bias: bool,
        omega_0: bool,
        dropout: float,
        weight_dropout: float,
    ):
        super().__init__(
            in_channels,
            hidden_channels,
            num_blocks,
            kernelnet_hidden_channels,
            kernelnet_activation_function,
            kernelnet_norm_type,
            dim_linear,
            bias,
            omega_0,
            dropout,
            weight_dropout,
        )

        self.finallyr = torch.nn.Linear(
            in_features=hidden_channels, out_features=out_channels
        )
        # Initialize finallyr
        self.finallyr.weight.data.normal_(
            mean=0.0,
            std=0.01,
        )
        self.finallyr.bias.data.fill_(value=0.0)

    def forward(self, x):
        out = self.backbone(x)
        out = self.finallyr(out[:, :, -1])
        return out

In [162]:
# computes the test loss per epoch for train and validation
# since we're not training, we don't need to calculate the gradients for our outputs
def testLoss(network,n_test,batch_size,X1_test,Y1_test ):
    criterion = nn.MSELoss().to(device)    
    running_test_loss=0.0
    epoch_test_loss = 0.0
    
    with torch.no_grad():
        for i in np.arange(0,n_test,batch_size):

            test_inputs = X1_test[i:i+batch_size].cuda()
            test_true_val = Y1_test[i:i+batch_size].cuda()

            # forward + backward + optimize
            test_outputs = network(test_inputs).to(device)
            test_loss = criterion(test_outputs.to(device), test_true_val.to(device))

            running_test_loss += test_loss.item()            
            if batch_size != len(test_inputs):
                epoch_test_loss += running_test_loss* len(test_inputs)
            else:                
                epoch_test_loss += running_test_loss* batch_size
            running_test_loss = 0.0
    
    return epoch_test_loss/n_test

In [163]:
def modelCkcnn(X1_train,Y1_train,X1_test,Y1_test,lr,units,hidden_units,batch,epochs = None, callback = False):
    print(units)
    print(hidden_units)
    print(epochs)
    # Construct network:
    in_channels = 5
    out_channels = 1
    hidden_channels = units
    num_blocks = 2
    kernelnet_hidden_channels = hidden_units
    kernelnet_activation_function = 'Sine'
    kernelnet_norm_type = ''
    dim_linear = 1
    bias = True
    omega_0 = 30.5
    dropout = 0
    weight_dropout = 0.0

    network = CKCNN(in_channels,
                    out_channels,
                    hidden_channels,
                    num_blocks,
                    kernelnet_hidden_channels,
                    kernelnet_activation_function,
                    kernelnet_norm_type,
                    dim_linear,
                    bias,
                    omega_0,
                    dropout,
                    weight_dropout,
                   )
    network.to(device)
    
    
    loss = []
    loss_change =[]
    total_loss = []
    total_val_loss = []
    patience = 5
    patience_change = patience
    batch_size = batch
    criterion = nn.MSELoss().to(device)
    #optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
    optimizer = optim.Adam(network.parameters(), lr=lr)
    if epochs == None:
        epochs =250
    n_train = len(X1_train)
    n_test = len(X1_test)

    for epoch in range(epochs):  # loop over the dataset multiple times
        running_loss = 0.0
        epoch_loss = 0.0
        count = 0
        
        for i in np.arange(0,n_train,batch_size):
            inputs = X1_train[i:i+batch_size].cuda()
            true_val = Y1_train[i:i+batch_size].cuda()

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = network(inputs).to(device)
            loss = criterion(outputs.cuda(), true_val.cuda())
            loss.backward()
            optimizer.step()

             # print statistics
            running_loss += loss.item()
            if batch_size != len(inputs):
                epoch_loss += running_loss * len(inputs)
            else:
                epoch_loss += running_loss * batch_size
            running_loss = 0.0


        train_loss =   epoch_loss/n_train 
        total_loss.append(train_loss)
        outputs = network(X1_test[:].cuda())
        test_loss = testLoss(network,n_test,batch_size,X1_test,Y1_test )
        total_val_loss.append(test_loss)

        if callback == True:
            loss_change.append(abs(train_loss - test_loss))
            if len(loss_change)>patience:
                if np.mean(loss_change[-patience:])> prev_mean:
                    patience_change -= 1
                    if patience_change==0:
                        break
                else:
                    patience_change = patience
            prev_mean = np.mean(loss_change[-patience:])
        print("Epoch %d/%d: loss - %f, val loss - %f" % (epoch+1, epochs, train_loss, test_loss) )
    r = [total_loss,total_val_loss]

    return r,network

    

## Results and Prediction Generators <a name="7"></a>

In [250]:
# generates the results for the tesorflow models, includes the following steps:
# 1) preprecessing
# 2) train/test splitting
# 3) model training
# 4) model validation 
# returns the test/train mse and the predictions
def resultsGenerator(input_data, offset_list, m,lr,units,batch,epochs,callback):
    
    if callback == True:
        epochs=250
    extended_model_info  = pd.DataFrame()
    
    for target_offset in offset_list:        
        model_dict = {}
        model_dict["Target Step Offset"] = target_offset
        X, Y , X_unscaled, Y_unscaled, scaler_list = preprocessor(input_data,target_offset)
        X_train,Y_train, X_test, Y_test, X_train_unscaled, Y_train_unscaled, X_test_unscaled, Y_test_unscaled = trainTestSplit(
                                                                                        ratio ,X, Y , X_unscaled, Y_unscaled)                
        if m == "Attention":
            r,model = attentionGRU(epochs,X_train, Y_train , X_test, Y_test,lr=lr,units=units,
                                   batch=batch,callback = callback,verbose = 1, plot_summary = False)        
        elif m =="GRU":
            r,model = modelGRU(epochs,X_train, Y_train , X_test, Y_test,lr=lr,units=units,
                                   batch=batch,callback = callback,verbose = 1, plot_summary = False)            
        elif m =="LSTM":
            r,model = modelLSTM(epochs,X_train, Y_train , X_test, Y_test,lr=lr,units=units,
                                   batch=batch,callback = callback,verbose = 1, plot_summary = False)            
        else:
            r,model = modelRNN(epochs,X_train, Y_train , X_test, Y_test,lr=lr,units=units,
                                   batch=batch,callback = callback,verbose = 1, plot_summary = False)

        mse = tf.keras.losses.MeanSquaredError()

        outputs_test = model.predict(X_test)
        outputs_train = model.predict(X_train)

        model_dict["Train MSE"] = float(mse(Y_train, outputs_train )  ) 
        model_dict["Test MSE"] = float(mse(Y_test, outputs_test )  ) 

        model_dict["Learning Train MSE"] = r.history['loss']
        model_dict["Learning Test MSE"] = r.history['val_loss']

        train_length = len(X_train) 
        close_predictions = getPrediction(model, X_test, Y_test, Y_test_unscaled, scaler_list[train_length:])
        model_dict["Test Predictions"] = close_predictions        
        model_dict["Test True Value"] = Y_test_unscaled       
        model_dict["Test Previous Day"] = X_test_unscaled[:,-1,3]
        close_predictions = getPrediction(model ,X_train, Y_train, Y_train_unscaled, scaler_list[:train_length])

        model_dict["Train Predictions"] = close_predictions
        model_dict["Train True Value"] = Y_train_unscaled
        model_dict["Train Previous Day"] = X_train_unscaled[:,-1,3]
        model_dict["Train Length = "] = train_length

        extended_model_info = extended_model_info.append(model_dict, ignore_index=True)
    return extended_model_info

In [164]:
# generate results for the ckcnn (pytorch)
# 1) preprecessing
# 2) train/test splitting
# 3) model training
# 4) model validation 
# returns the test/train mse and the predictions
def resultsGenerator2 (input_data, offset_list,units,lr,hidden_units,batch,callback,epochs):

    extended_model_info  = pd.DataFrame()
    criterion = nn.MSELoss().to(device) 
    for target_offset in offset_list:
        model_dict = {}
        print("$$$$$$$$  pytorch $$$$$$$$$")

        model_dict["Target Step Offset"] = target_offset


        X, Y , X_unscaled, Y_unscaled, scaler_list = preprocessor(input_data,target_offset)
        X_train,Y_train, X_test, Y_test, X_train_unscaled, Y_train_unscaled, X_test_unscaled, Y_test_unscaled = trainTestSplit(
                                                                                        ratio ,X, Y , X_unscaled, Y_unscaled)
        n_train = len(X_train)
        X1_train = np.zeros((n_train,input_D, T))
        Y1_train = np.zeros((n_train,output_D))
        X1_train_unscaled = np.zeros((n_train,input_D, T))
        Y1_train_unscaled  = np.zeros((n_train,output_D))


        n_test = len(X_test)
        X1_test = np.zeros((n_test,input_D, T))
        Y1_test = np.zeros((n_test,output_D))
        X1_test_unscaled = np.zeros((n_test,input_D, T))
        Y1_test_unscaled  = np.zeros((n_test,output_D))


        for i in range(len(X)):
            if i <n_train:
                X1_train[i]= X_train[i].T
                Y1_train[i]= Y_train[i]
                X1_train_unscaled[i] = X_train_unscaled[i].T
                Y1_train_unscaled[i] = Y_train_unscaled[i]
                
                
            if i < n_test:
                X1_test[i]= X_test[i].T
                Y1_test[i]= Y_test[i]
                X1_test_unscaled[i] = X_test_unscaled[i].T
                Y1_test_unscaled[i] = Y_test_unscaled[i]
                


        X1_train =torch.tensor( X1_train).float()
        Y1_train = torch.tensor(Y1_train).float()

        X1_test = torch.tensor(X1_test).float()
        Y1_test = torch.tensor(Y1_test).float()
       
        r,model = modelCkcnn(X1_train,Y1_train,X1_test,Y1_test,lr,units,
                             hidden_units,batch,epochs, callback )
        X1_train = X1_train.to(device)
        Y1_train = Y1_train.to(device)
        X1_test = X1_test.to(device)
        Y1_test = Y1_test.to(device)
        
        outputs_test = model(X1_test)
        outputs_train = model(X1_train)
        mse = tf.keras.losses.MeanSquaredError()

        model_dict["Train MSE"] =  criterion(outputs_train.to(device), Y1_train).item()
        model_dict["Test MSE"] =  criterion(outputs_test.to(device),  Y1_test).item()


        model_dict["Learning Train MSE"] = r[0]
        model_dict["Learning Test MSE"] = r[1]
        
        close_predictions = getPrediction2(model, X1_test, Y1_test, Y1_test_unscaled, scaler_list[n_train:])



        model_dict["Test Predictions"] = close_predictions
        print("Test Predictions")
        print(close_predictions.shape)
        model_dict["Test True Value"] = Y1_test_unscaled
        print("Test True Value")
        print(Y_test_unscaled.shape)
        model_dict["Test Previous Day"] = X1_test_unscaled[:,3,-1]
        print("Test Previous Day")
        print(X_test_unscaled.shape)



        close_predictions = getPrediction2(model ,X1_train, Y1_train, Y1_train_unscaled, scaler_list[:n_train])

        model_dict["Train Predictions"] = close_predictions
        model_dict["Train True Value"] = Y1_train_unscaled
        model_dict["Train Previous Day"] = X1_train_unscaled[:,-3,-1]
        model_dict["Train Length = "] = n_train


        extended_model_info = extended_model_info.append(model_dict, ignore_index=True)

    return extended_model_info

In [251]:
# Gets the prediction for the tensorflow models
def getPrediction(model , X,Y, Y_unscaled, scaler_list):
    predictions = model.predict(X)
    unscaled_predictions = np.zeros(len(predictions))
    temp =  np.zeros((T,input_D))

    for k in range(len(predictions)):
        temp[-1,3:4] = predictions[k]
        unscaled_temp=scaler_list[k].inverse_transform(temp)
        unscaled_prediction = unscaled_temp[-1,3:4]
        unscaled_predictions[k]=unscaled_prediction    
        
    return unscaled_predictions



# gets the predictions for the ckcnn model (pytorch)
def getPrediction2(model , X,Y, Y_unscaled, scaler_list):
    predictions = model(X.to(device))
    predictions = predictions.cpu().detach().numpy()
    unscaled_predictions = np.zeros(len(predictions))
    temp =  np.zeros((T,input_D))

    for k in range(len(predictions)):
        temp[-1,3:4] = predictions[k]        
        unscaled_temp=scaler_list[k].inverse_transform(temp)
        unscaled_prediction = unscaled_temp[-1,3:4]        
        unscaled_predictions[k]=unscaled_prediction
        
    return unscaled_predictions

In [249]:
# gets the the percent profit of a single prediction for long trades
def tradeLong(prediction, true_val, prev_close  ):
    profit = 0
    if prediction>prev_close:
        if prediction<true_val:
            profit = prediction - prev_close 
        else: 
            profit = true_val - prev_close
    return round(profit/prev_close,4)

# gets the the percent profit of a single prediction for short trades        
def tradeShort(prediction, true_val, prev_close  ):
    profit = 0
    if prediction<prev_close:
        if prediction>true_val:
            profit = prediction-prev_close
        else: 
            profit = prev_close - true_val
    return round(profit/prev_close,4)

# generates the performance of the predictions as well as the buy and hold benchmark
def performanceGenerator(extended_model_info,offset_list):
    for t in ["Train ","Test "]:
        long_performance = []
        short_performance = []
        Total_performance = []

        for target_offset in offset_list:
            eval_df = pd.DataFrame()
            eval_df["Prediction"] = extended_model_info[extended_model_info["Target Step Offset"]==target_offset][t+"Predictions"].values[0]
            eval_df["True Value"] =extended_model_info[extended_model_info["Target Step Offset"]==target_offset][t+"True Value"].values[0]
            eval_df["Previous Close"] = extended_model_info[extended_model_info["Target Step Offset"]==target_offset][t+"Previous Day"].values[0]
            eval_df["Long Performance"] = eval_df.apply(lambda x: tradeLong(x["Prediction"],x["True Value"]
                                                                            , x["Previous Close"]) ,axis = 1)
            eval_df["Short Performance"] = eval_df.apply(lambda x: tradeShort(x["Prediction"],x["True Value"]
                                                                             , x["Previous Close"]) ,axis = 1 )
            long_performance.append(eval_df["Long Performance"].mean()/target_offset)
            short_performance.append(eval_df["Short Performance"].mean()/target_offset)
            Total_performance.append((eval_df["Short Performance"]+eval_df["Long Performance"]).mean()/target_offset)


        extended_model_info[t+"Average Long Performance"] = np.array(long_performance)*100
        extended_model_info[t+"Average Short Performance"] = np.array(short_performance)*100
        extended_model_info[t+"Average Performance"] = np.array(Total_performance)*100

        first_close_price = extended_model_info[extended_model_info["Target Step Offset"]==offset_list[0]][t+"True Value"].values[0][0]
        last_close_price = extended_model_info[extended_model_info["Target Step Offset"]==offset_list[0]][t+"True Value"].values[0][-1]
        
        
        extended_model_info[t+"Buy and Hold"] = 100*((last_close_price[0] - first_close_price[0])/first_close_price[0])/(len(eval_df["Prediction"])-1)

    return extended_model_info

## Data Learner <a name="8"></a>

In [166]:
# Learns the data on a given list of symbols
# params: 
# symbols - a alist of stocks
# model_parameters - a dataframe returns from the optimizer, includes the lr,batch,epochs, 
# callbacl -  true for stopping the learning with accordance to the test mse, false for no intervention
def learner(symbols,model_parameters,callback):    
    temp_m = np.array(model_parameters.index.get_level_values('Model'))
    models = np.sort(np.unique(temp_m)).tolist()
        
    temp_o = np.array(model_parameters.index.get_level_values('Offset'),dtype = np.int32)
    offset_list = np.sort(np.unique(temp_o)).tolist()
    
    reports = ["Target Step Offset","Train MSE","Test MSE","Train Average Performance","Test Average Performance",
                    "Train Average Long Performance","Train Average Short Performance",
                    "Test Average Long Performance","Test Average Short Performance",
                                      "Train Buy and Hold","Test Buy and Hold"]

    extended_report = ["Learning Train MSE","Learning Test MSE","Test Predictions","Test True Value",
                       "Test Previous Day","Train Predictions","Train True Value","Train Previous Day"]
    reports_matrix = np.zeros((len(symbols),len(models),len(offset_list),len(reports)))

    extended_reports_matrix =[]
    symbol_models = dict.fromkeys(symbols)
    extended_reports_symbol =[]
    for symbol in symbols:
        stock = df[df['symbol']==symbol]
        stock = stock.sort_values(by = "date")
        input_data = stock[['open', 'high', 'low', 'close', 'volume']].values
        n = len(input_data)
        extended_reports_model =[]
        for model in models:
            extended_reports_offset=[]
            for offset in offset_list:
                
                # Parameters
                lr = model_parameters.loc[model,offset]["lr"]
                units = int(model_parameters.loc[model,offset]["units"])
                batch = int(model_parameters.loc[model,offset]["batch"])
                epochs = int(model_parameters.loc[model,offset]["epochs"])

                if model!="CKCNN":  
                    model_info = resultsGenerator(input_data, [offset],m = model,callback = callback,
                                                    units=units,lr=lr,batch= batch,epochs= epochs)
                else:
                    hidden_units =int( model_parameters.loc[model,offset]["hidden units"])
                    model_info = resultsGenerator2(input_data, [offset],units=units,callback = callback,hidden_units=hidden_units,lr=lr,batch= batch,epochs= epochs)
                    

                extended_model_info = performanceGenerator(model_info,[offset])

                critical_reports = extended_model_info[reports].to_numpy()
                extended_critical_reports = extended_model_info[extended_report].to_numpy()

                reports_matrix[symbols.index(symbol),
                              models.index(model),
                              offset_list.index(offset)]=critical_reports

                extended_reports_offset.append(extended_critical_reports)
            extended_reports_model.append(extended_reports_offset)
        extended_reports_matrix.append(extended_reports_model)
        
    return [np.array(reports_matrix),np.array(extended_reports_matrix)]

## Hyper Parameter Tuning

In [167]:

# find the hyper parameters: lr, units, and batch
def optimizer(offset_list,models= ["LSTM","GRU","RNN","Attention","CKCNN"]):
    
    symbols =  ["INTC","MSFT","DAL","MMM","JNJ","PFE","AIG","JPM","XOM"] 
    
    reports = ["Target Step Offset","Train MSE","Test MSE","Train Average Performance","Test Average Performance",
                "Train Average Long Performance","Train Average Short Performance",
                "Test Average Long Performance","Test Average Short Performance",
                                  "Train Buy and Hold","Test Buy and Hold"]

    extended_report = ["Learning Train MSE","Learning Test MSE","Test Predictions","Test True Value",
                   "Test Previous Day","Train Predictions","Train True Value","Train Previous Day"]
    
    
    hyper_param =  pd.DataFrame()
    units_list=[7,10,15,20,30]
    lr_list = [0.1,0.05,0.01,0.005,0.001]
    batch_list = [32]
    
    for offset in offset_list:
        for units in units_list:
            for lr in lr_list:
                for batch in batch_list:
                    for model in models:
                        print("%%%%%%%%%%%%%%%%%%%%%%%%%%%")
                        print([offset,units,lr,batch,model])
                        model_p = pd.DataFrame()
                        model_p = model_p.append({"Model":model,"Offset":offset,"units":units,"hidden units":units,"lr":lr
                                        ,"batch":batch,"epochs":250},ignore_index=True)
                        model_p = model_p.set_index(['Model','Offset'])
                        output = learner(symbols,model_p,callback = True)
                        reports_matrix = output[0]
                        extended_reports_matrix = output[1]
                    
                        epochs = 0

                        for i in range(len(symbols)):
                            epochs+=len(extended_reports_matrix[:,0,0,0,extended_report.index("Learning Train MSE")][0])  
                        test_mse = np.mean(reports_matrix[:,0,0,reports.index("Test MSE")])
                        epochs = epochs/len(symbols)
                        hyper_param = hyper_param.append({"Model":model,"Offset":offset,"units":units
                                        ,"hidden units":units,"lr":lr,"batch":batch,"epochs":epochs
                                        ,"Test MSE":test_mse},ignore_index=True)  
                        
    hyper_param.to_csv("all_parameters.csv")  
    return hyper_param
   

In [ ]:


%%time
parameter_tables = optimizer( [1,2,3,4,5])

all_params = pd.read_csv('all_parameters.csv')  
best_params= all_params[['Model', 'Offset','Test MSE']].groupby(['Model', 'Offset']).min()
best_params = best_params.merge(all_params,on = ['Test MSE'])
best_params =best_params.set_index(['Model','Offset'])

In [219]:
# find the hyper parameters
all_params = pd.read_csv('all_parameters.csv')  
best_params=all_params[['Model', 'Offset','Test MSE']].groupby(['Model', 'Offset']).min()
best_params = best_params.merge(all_params,on = ['Test MSE'])
best_params =best_params.set_index(['Model','Offset'])
best_params.to_csv("best_params.csv")

In [236]:
# learn the data of the given stocks with the given given hyper parameters
symbols = ['QCOM','NKE','MCD','GRMN','CERN','AAPL','SCHW',"CVX","EBAY"]
results =learner(symbols,best_params,callback = False)

Epoch 1/47
16/16 [==============================] - 0s 28ms/step - loss: 0.1290 - val_loss: 0.1058
Epoch 2/47
16/16 [==============================] - 0s 9ms/step - loss: 0.1133 - val_loss: 0.0839
Epoch 3/47
16/16 [==============================] - 0s 9ms/step - loss: 0.0952 - val_loss: 0.0817
Epoch 4/47
16/16 [==============================] - 0s 9ms/step - loss: 0.0913 - val_loss: 0.0746
Epoch 5/47
16/16 [==============================] - 0s 9ms/step - loss: 0.0851 - val_loss: 0.0750
Epoch 6/47
16/16 [==============================] - 0s 9ms/step - loss: 0.0841 - val_loss: 0.0714
Epoch 7/47
16/16 [==============================] - 0s 9ms/step - loss: 0.0851 - val_loss: 0.0709
Epoch 8/47
16/16 [==============================] - 0s 9ms/step - loss: 0.0766 - val_loss: 0.0695
Epoch 9/47
16/16 [==============================] - 0s 9ms/step - loss: 0.0783 - val_loss: 0.0764
Epoch 10/47
16/16 [==============================] - 0s 9ms/step - loss: 0.0797 - val_loss: 0.0668
Epoch 11/47
16/16 

Epoch 37/45
16/16 [==============================] - 0s 9ms/step - loss: 0.0787 - val_loss: 0.0758
Epoch 38/45
16/16 [==============================] - 0s 9ms/step - loss: 0.0812 - val_loss: 0.0811
Epoch 39/45
16/16 [==============================] - 0s 9ms/step - loss: 0.0823 - val_loss: 0.0763
Epoch 40/45
16/16 [==============================] - 0s 9ms/step - loss: 0.0796 - val_loss: 0.0767
Epoch 41/45
16/16 [==============================] - 0s 9ms/step - loss: 0.0799 - val_loss: 0.0767
Epoch 42/45
16/16 [==============================] - 0s 9ms/step - loss: 0.0819 - val_loss: 0.0775
Epoch 43/45
16/16 [==============================] - 0s 9ms/step - loss: 0.0802 - val_loss: 0.0774
Epoch 44/45
16/16 [==============================] - 0s 9ms/step - loss: 0.0785 - val_loss: 0.0785
Epoch 45/45
16/16 [==============================] - 0s 9ms/step - loss: 0.0778 - val_loss: 0.0785
Epoch 1/23
16/16 [==============================] - 0s 30ms/step - loss: 0.1652 - val_loss: 0.1255
Epoch 2/23

Epoch 29/31
16/16 [==============================] - 0s 6ms/step - loss: 0.0876 - val_loss: 0.0983
Epoch 30/31
16/16 [==============================] - 0s 6ms/step - loss: 0.0870 - val_loss: 0.0982
Epoch 31/31
16/16 [==============================] - 0s 6ms/step - loss: 0.0880 - val_loss: 0.0978
$$$$$$$$  pytorch $$$$$$$$$
30
30
15
Epoch 1/15: loss - 0.842594, val loss - 0.136894
Epoch 2/15: loss - 0.194887, val loss - 0.099302
Epoch 3/15: loss - 0.150352, val loss - 0.073706
Epoch 4/15: loss - 0.086379, val loss - 0.087800
Epoch 5/15: loss - 0.083232, val loss - 0.081822
Epoch 6/15: loss - 0.080340, val loss - 0.069874
Epoch 7/15: loss - 0.075947, val loss - 0.068366
Epoch 8/15: loss - 0.073807, val loss - 0.065466
Epoch 9/15: loss - 0.070452, val loss - 0.064932
Epoch 10/15: loss - 0.070734, val loss - 0.063398
Epoch 11/15: loss - 0.068852, val loss - 0.063759
Epoch 12/15: loss - 0.068492, val loss - 0.063013
Epoch 13/15: loss - 0.065993, val loss - 0.063290
Epoch 14/15: loss - 0.065

Epoch 139/176: loss - 0.060577, val loss - 0.095867
Epoch 140/176: loss - 0.059173, val loss - 0.122469
Epoch 141/176: loss - 0.062099, val loss - 0.111742
Epoch 142/176: loss - 0.065095, val loss - 0.120519
Epoch 143/176: loss - 0.063205, val loss - 0.104821
Epoch 144/176: loss - 0.059378, val loss - 0.100512
Epoch 145/176: loss - 0.061639, val loss - 0.110854
Epoch 146/176: loss - 0.062959, val loss - 0.110446
Epoch 147/176: loss - 0.063791, val loss - 0.133408
Epoch 148/176: loss - 0.061851, val loss - 0.105172
Epoch 149/176: loss - 0.061564, val loss - 0.111064
Epoch 150/176: loss - 0.061333, val loss - 0.120327
Epoch 151/176: loss - 0.061578, val loss - 0.120027
Epoch 152/176: loss - 0.062279, val loss - 0.102374
Epoch 153/176: loss - 0.060878, val loss - 0.119819
Epoch 154/176: loss - 0.064835, val loss - 0.130977
Epoch 155/176: loss - 0.066450, val loss - 0.128795
Epoch 156/176: loss - 0.065071, val loss - 0.113050
Epoch 157/176: loss - 0.064475, val loss - 0.109927
Epoch 158/17

16/16 [==============================] - 0s 2ms/step - loss: 0.0967 - val_loss: 0.0763
Epoch 3/28
16/16 [==============================] - 0s 2ms/step - loss: 0.0835 - val_loss: 0.0721
Epoch 4/28
16/16 [==============================] - 0s 2ms/step - loss: 0.0770 - val_loss: 0.0740
Epoch 5/28
16/16 [==============================] - 0s 2ms/step - loss: 0.0697 - val_loss: 0.0729
Epoch 6/28
16/16 [==============================] - 0s 3ms/step - loss: 0.0622 - val_loss: 0.0753
Epoch 7/28
16/16 [==============================] - 0s 2ms/step - loss: 0.0596 - val_loss: 0.0693
Epoch 8/28
16/16 [==============================] - 0s 3ms/step - loss: 0.0607 - val_loss: 0.0700
Epoch 9/28
16/16 [==============================] - 0s 3ms/step - loss: 0.0571 - val_loss: 0.0659
Epoch 10/28
16/16 [==============================] - 0s 3ms/step - loss: 0.0540 - val_loss: 0.0655
Epoch 11/28
16/16 [==============================] - 0s 3ms/step - loss: 0.0539 - val_loss: 0.0643
Epoch 12/28
16/16 [==========

...

Epoch 87/138
16/16 [==============================] - 0s 2ms/step - loss: 0.0488 - val_loss: 0.0796
Epoch 88/138
16/16 [==============================] - 0s 2ms/step - loss: 0.0494 - val_loss: 0.0722
Epoch 89/138
16/16 [==============================] - 0s 2ms/step - loss: 0.0475 - val_loss: 0.0720
Epoch 90/138
16/16 [==============================] - 0s 2ms/step - loss: 0.0469 - val_loss: 0.0731
Epoch 91/138
16/16 [==============================] - 0s 2ms/step - loss: 0.0476 - val_loss: 0.0728
Epoch 92/138
16/16 [==============================] - 0s 2ms/step - loss: 0.0479 - val_loss: 0.0726
Epoch 93/138
16/16 [==============================] - 0s 2ms/step - loss: 0.0471 - val_loss: 0.0721
Epoch 94/138
16/16 [==============================] - 0s 2ms/step - loss: 0.0463 - val_loss: 0.0722
Epoch 95/138
16/16 [==============================] - 0s 2ms/step - loss: 0.0458 - val_loss: 0.0727
Epoch 96/138
16/16 [==============================] - 0s 2ms/step - loss: 0.0462 - val_loss: 0.0734


16/16 [==============================] - 0s 2ms/step - loss: 0.1058 - val_loss: 0.1052
Epoch 31/149
16/16 [==============================] - 0s 2ms/step - loss: 0.1057 - val_loss: 0.1062
Epoch 32/149
16/16 [==============================] - 0s 2ms/step - loss: 0.1056 - val_loss: 0.1047
Epoch 33/149
16/16 [==============================] - 0s 2ms/step - loss: 0.1052 - val_loss: 0.1061
Epoch 34/149
16/16 [==============================] - 0s 2ms/step - loss: 0.1054 - val_loss: 0.1046
Epoch 35/149
16/16 [==============================] - 0s 2ms/step - loss: 0.1046 - val_loss: 0.1052
Epoch 36/149
16/16 [==============================] - 0s 2ms/step - loss: 0.1045 - val_loss: 0.1051
Epoch 37/149
16/16 [==============================] - 0s 2ms/step - loss: 0.1043 - val_loss: 0.1045
Epoch 38/149
16/16 [==============================] - 0s 2ms/step - loss: 0.1040 - val_loss: 0.1044
Epoch 39/149
16/16 [==============================] - 0s 2ms/step - loss: 0.1040 - val_loss: 0.1049
Epoch 40/149


Epoch 112/149
16/16 [==============================] - 0s 2ms/step - loss: 0.0689 - val_loss: 0.0883
Epoch 113/149
16/16 [==============================] - 0s 2ms/step - loss: 0.0686 - val_loss: 0.0889
Epoch 114/149
16/16 [==============================] - 0s 2ms/step - loss: 0.0686 - val_loss: 0.0886
Epoch 115/149
16/16 [==============================] - 0s 2ms/step - loss: 0.0683 - val_loss: 0.0890
Epoch 116/149
16/16 [==============================] - 0s 2ms/step - loss: 0.0680 - val_loss: 0.0886
Epoch 117/149
16/16 [==============================] - 0s 2ms/step - loss: 0.0692 - val_loss: 0.0905
Epoch 118/149
16/16 [==============================] - 0s 2ms/step - loss: 0.0678 - val_loss: 0.0883
Epoch 119/149
16/16 [==============================] - 0s 2ms/step - loss: 0.0678 - val_loss: 0.0888
Epoch 120/149
16/16 [==============================] - 0s 2ms/step - loss: 0.0672 - val_loss: 0.0885
Epoch 121/149
16/16 [==============================] - 0s 2ms/step - loss: 0.0672 - val_los

16/16 [==============================] - 0s 2ms/step - loss: 0.0619 - val_loss: 0.0948
Epoch 46/56
16/16 [==============================] - 0s 2ms/step - loss: 0.0633 - val_loss: 0.0922
Epoch 47/56
16/16 [==============================] - 0s 2ms/step - loss: 0.0611 - val_loss: 0.0976
Epoch 48/56
16/16 [==============================] - 0s 2ms/step - loss: 0.0582 - val_loss: 0.0929
Epoch 49/56
16/16 [==============================] - 0s 2ms/step - loss: 0.0587 - val_loss: 0.0943
Epoch 50/56
16/16 [==============================] - 0s 2ms/step - loss: 0.0591 - val_loss: 0.0953
Epoch 51/56
16/16 [==============================] - 0s 2ms/step - loss: 0.0610 - val_loss: 0.0953
Epoch 52/56
16/16 [==============================] - 0s 2ms/step - loss: 0.0588 - val_loss: 0.0963
Epoch 53/56
16/16 [==============================] - 0s 2ms/step - loss: 0.0578 - val_loss: 0.1031
Epoch 54/56
16/16 [==============================] - 0s 2ms/step - loss: 0.0602 - val_loss: 0.0946
Epoch 55/56
16/16 [===

16/16 [==============================] - 0s 2ms/step - loss: 0.0830 - val_loss: 0.1097
Epoch 13/41
16/16 [==============================] - 0s 2ms/step - loss: 0.0795 - val_loss: 0.1090
Epoch 14/41
16/16 [==============================] - 0s 2ms/step - loss: 0.0768 - val_loss: 0.1080
Epoch 15/41
16/16 [==============================] - 0s 2ms/step - loss: 0.0766 - val_loss: 0.1059
Epoch 16/41
16/16 [==============================] - 0s 2ms/step - loss: 0.0753 - val_loss: 0.1051
Epoch 17/41
16/16 [==============================] - 0s 2ms/step - loss: 0.0751 - val_loss: 0.1124
Epoch 18/41
16/16 [==============================] - 0s 2ms/step - loss: 0.0742 - val_loss: 0.1039
Epoch 19/41
16/16 [==============================] - 0s 2ms/step - loss: 0.0727 - val_loss: 0.1100
Epoch 20/41
16/16 [==============================] - 0s 2ms/step - loss: 0.0751 - val_loss: 0.1033
Epoch 21/41
16/16 [==============================] - 0s 2ms/step - loss: 0.0734 - val_loss: 0.1028
Epoch 22/41
16/16 [===

In [237]:
# get the reports from the leaner and save it locally
reports_matrix = results[0]
extended_reports_matrix = results[1]
np.save('reports.npy', reports_matrix) # save
np.save('extended reports.npy', np.array(extended_reports_matrix,dtype=object)) # save